# **11.3 2D Homographies - Applications**

Now you are familiar with homographies, this notebook proposes two more applications in sports where using this technique enhances the experience of viewer. You are asked to implement a solution for each one. As you already know the fundamentals of the homography computation, this notebook will not address any theoretical aspect, but only presents the applications and asks for your solutions!

### Some notes on the implementation
- If you use the command `%matplotlib notebook`, you'll be able to get the coordinates on the image at the mouse position. Use this **in each cell** when you need to select points on the images. Unfortunately this doesn't work in all jupyter frameworks (not in Google Colab, for instance). If it does not work, you'll have to get the point coordinates by opening the image with a typical image processing application (such as Irfanview, GIMP, etc.).
- Take into account that when you are computing a homography you are mapping image coordinates to *real world* coordinates (those in the real, unwarped plane), so if the latter **are too small** you will end up with a very small unwarped image. This will make more difficult to get references in such a small image. In some way, you are *losing resolution*. So, when creating the coordinates of the unwarped planes, use a value (factor) to multiply the real coordinates so that you get an image large enough to pick up points properly.

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)

# set image path
images_path = '/gdrive/My Drive/Colab Notebooks/Chapter 11. Real-world applications/images/'

# This will allow you to see the pixel position where the mouse is pointing
# %matplotlib notebook
%matplotlib inline

## **11.3.1 Application 1: Long jump competiton**

The *Olympics committee* has a special work for you: to develop a computer vision program able to decide if a jump is valid (the jumper has not overstepped the plasticine board) and the distance from the step to this plasticine board.

The following image shows the parts of the zone where the jumper has to take-off: the **take-off board** (with a width of 20cm), the **take-off line** (included in the take-off board) and the **plasticine indicator** (10cm width). In total, the area of this zone is **30$\times$122cm**.

<center>
<img src="https://raw.githubusercontent.com/famoreno/cv_jn_images/master/ch8/insert/take_off_zone.png" width="300">$\\[5pt]$
</center>

The *Olympics committee* wants you to evaluate your software using a number of provided images, named `long_jump_0.png` to `lon_jump_4.png`, which are annotated with the distance from the jumper sneaker tip to the beginning of the plasticine indicator. With this goal, implement a code that:
- Computes the homography between the take-off zone in the images and a rectangle with the real size (30x122cm).
- Select manually the point at the jumper sneaker tip in the zone without perspective.
- Having that coordinate, computes and reports the distance from it to the plasticine indicator.

Is this distance the same as the provided one in the images?

In [ ]:
# APPLICATION 1: Long jump competition
# Write your code here

# Load the image and show it
# Get the proper points in the image and plot a circle on them (in a new image)
# Show this new image
# Find the homography and remove the perspective
# Compute the distance (in centimeters) between the sneaker and the indicator
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
%matplotlib notebook

# APPLICATION 1: Long jump competition
image1=cv2.imread(images_path + 'long_jump_0.png', -1)
image2=cv2.imread(images_path + 'long_jump_1.png', -1)
image3=cv2.imread(images_path + 'long_jump_2.png', -1)
image4=cv2.imread(images_path + 'long_jump_3.png', -1)
image5=cv2.imread(images_path + 'long_jump_4.png', -1)

image1=cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
image2=cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
image3=cv2.cvtColor(image3, cv2.COLOR_BGR2RGB)
image4=cv2.cvtColor(image4, cv2.COLOR_BGR2RGB)
image5=cv2.cvtColor(image5, cv2.COLOR_BGR2RGB)
offset=1000
points1 = np.asarray([(474, 57), (150, 535),(697, 55),(789, 554)])
points2 = np.asarray([(offset,offset),(offset, offset+1220), (offset+300,offset), (offset+300,offset+1220)])
M = cv2.findHomography(points1, points2)
transformed1 = cv2.warpPerspective(image1, M[0], (2*image1.shape[1],4*image1.shape[0]))
transformed2 = cv2.warpPerspective(image2, M[0], (2*image1.shape[1],4*image1.shape[0]))
transformed3 = cv2.warpPerspective(image3, M[0], (2*image1.shape[1],4*image1.shape[0]))
transformed4 = cv2.warpPerspective(image4, M[0], (2*image1.shape[1],4*image1.shape[0]))
transformed5 = cv2.warpPerspective(image5, M[0], (2*image1.shape[1],4*image1.shape[0]))
plt.subplot(231)
plt.imshow(transformed1)
plt.subplot(232)
plt.imshow(transformed2)
plt.subplot(233)
plt.imshow(transformed3)
plt.subplot(234)
plt.imshow(transformed4)
plt.subplot(235)
plt.imshow(transformed5)
#Image1
p1x = (1167) # Punta del zapato
p2x = (1201) # Plastilina
pix2cm = 20/(1220-1000)
distancia_image1=(p2x-p1x)*pix2cm
print(distancia_image1) # Distancia = 3.09. (3 en la imagen)

#Image2
p1x2 = (1190) # Punta del zapato
distancia_image2=(p2x-p1x2)*pix2cm
print(distancia_image2) # Distancia = 1.0. (1 en la imagen)

#Image3
p1x3 = (1083) # Punta del zapato
distancia_image3=(p2x-p1x3)*pix2cm
print(distancia_image3) # Distancia = 10.72. (11 en la imagen)

#Image4
p1x4=(1102)
distancia_image4=(p2x-p1x4)*pix2cm
print(distancia_image4) # Distancia = 9.0. (9 en la imagen)

#Image5
p1x5 = (1156)
distancia_image5=(p2x-p1x5)*pix2cm
print(distancia_image5) # Distancia = 4.09. (4 en la imagen)

#### **<font color=red><b>Expected output</b></font>**

<img src="https://raw.githubusercontent.com/famoreno/cv_jn_images/master/ch11/insert/ch113_a1_result.png" width="300" />

## **11.3.2 Application 2: Offside detection**

*LaLiga* has also called to our offices asking for a software to help the VAR and to draw a line at the position of the last defender in a soccer match, making easier in this way to decide if a player was offside.

<center>
<img src="https://raw.githubusercontent.com/famoreno/cv_jn_images/master/ch8/insert/offside.jpg" width="600">$\\[5pt]$
</center>

To fulfill this requirement, develop a code that uses the `offside.jpg` image and:
- Removes the perspective of the penalty area. The size of this area is 16.5x40.3m.
- Gets (manually) two points at the positions of the last defender and the forward.
- Draws a line in those positimage crossing that position.

Now you have the right tools, could you tell whether the goal by Luis Suarez goal is legal? Can you determine the distance in meters between the lines for the defender and the forward?

In [ ]:
# APPLICATION 2: Offside detection
# Write your code here
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
%matplotlib notebook
# Load the image and show it
# Get the proper points in the image and plot a circle on them (in a new image)
# Show this new image
# Find the homography and remove the perspective
# Draw the offside line on the last defender position
# Draw the attacker line on the attacker position
# Wrap the image again to show both lines with perspective
# Compute the distance (in meters) between the two lines

image1=cv2.imread(images_path + 'offside.jpg', -1)
image1=cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
offset=200
points1 = np.asarray([(351, 28), (172, 355),(684, 40),(700, 380)])
points2 = np.asarray([(offset,offset),(offset, offset+403), (offset+165,offset), (offset+165,offset+403)])
M = cv2.findHomography(points1, points2)
transformed1 = cv2.warpPerspective(image1, M[0], (600,800))

Minv=np.linalg.inv(M[0])
#Linea
p1 = (240,0)
p2 = (240,800)

lines1=cv2.line(transformed1,p1,p2,(255, 0, 0),1)
transformed2 = cv2.warpPerspective(lines1, Minv, (image1.shape[1],image1.shape[0]))
plt.imshow(transformed2)

#### **<font color=red><b>Expected output</b></font>**

<img src="https://raw.githubusercontent.com/famoreno/cv_jn_images/master/ch11/insert/ch113_a2_result.png" width="800" />